In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import pandas as pd
import numpy as np
import time

<IPython.core.display.Javascript object>

In [6]:
# load in scraped tweets from https://github.com/tarrantcarter/Final_Capstone/blob/main/Modern_Motivation_Web_Scrape.ipynb
tweets = pd.read_csv(
    r"/Users/tarrantcarter/Dropbox/Data Science/Final_Capstone/motivational_tweets.csv"
)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,10,11,12,17,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<IPython.core.display.Javascript object>

In [7]:
tweets.head()

,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/LewisHowes/status/13509295...,2021-01-17 22:14:40+00:00,@businessbarista The School of Greatness has h...,@businessbarista The School of Greatness has h...,1350929560020410372,"{'username': 'LewisHowes', 'displayname': 'Lew...",[],[],1.0,0.0,...,0,1350896644489940992,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,"[{'username': 'businessbarista', 'displayname'..."
1,https://twitter.com/LewisHowes/status/13509292...,2021-01-17 22:13:17+00:00,Know this. Everything is happening for a reaso...,Know this. Everything is happening for a reaso...,1350929208520929282,"{'username': 'LewisHowes', 'displayname': 'Lew...",[],[],4.0,11.0,...,1,1350929208520929282,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,NaN
2,https://twitter.com/LewisHowes/status/13506630...,2021-01-17 04:35:28+00:00,@emilsterndorff @awilkinson @supercast @thecap...,@emilsterndorff @awilkinson @supercast @thecap...,1350663002639400960,"{'username': 'LewisHowes', 'displayname': 'Lew...",[],[],0.0,0.0,...,0,1350132120094556160,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,"[{'username': 'emilsterndorff', 'displayname':..."
3,https://twitter.com/LewisHowes/status/13503847...,2021-01-16 10:09:58+00:00,@theclaudmon Thank you!,@theclaudmon Thank you!,1350384793637314561,"{'username': 'LewisHowes', 'displayname': 'Lew...",[],[],1.0,0.0,...,0,1350062690539102209,en,"<a href=""http://www.zendesk.com"" rel=""nofollow...",http://www.zendesk.com,Zendesk,NaN,NaN,NaN,"[{'username': 'theclaudmon', 'displayname': 'C..."
4,https://twitter.com/LewisHowes/status/13503805...,2021-01-16 09:53:06+00:00,@MonsieurHolly Thank you!,@MonsieurHolly Thank you!,1350380547806289921,"{'username': 'LewisHowes', 'displayname': 'Lew...",[],[],0.0,0.0,...,0,1349879719446110209,en,"<a href=""http://www.zendesk.com"" rel=""nofollow...",http://www.zendesk.com,Zendesk,NaN,NaN,NaN,"[{'username': 'MonsieurHolly', 'displayname': ..."


<IPython.core.display.Javascript object>

In [8]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977162 entries, 0 to 977161
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   url              977162 non-null  object 
 1   date             977162 non-null  object 
 2   content          977155 non-null  object 
 3   renderedContent  977148 non-null  object 
 4   id               977148 non-null  object 
 5   user             977148 non-null  object 
 6   outlinks         977148 non-null  object 
 7   tcooutlinks      977148 non-null  object 
 8   replyCount       977148 non-null  float64
 9   retweetCount     977148 non-null  float64
 10  likeCount        977148 non-null  object 
 11  quoteCount       977148 non-null  object 
 12  conversationId   977148 non-null  object 
 13  lang             977148 non-null  object 
 14  source           977140 non-null  object 
 15  sourceUrl        977140 non-null  object 
 16  sourceLabel      977140 non-null  obje

<IPython.core.display.Javascript object>

In [14]:
# pd.set_option("")
pd.set_option("display.max_colwidth", None)

tweets["content"][:20]

0                                                                                                                                                                @businessbarista The School of Greatness has helped launched many million dollar brands but I’ll keep it the best kept secret 🤫 😝
1     Know this. Everything is happening for a reason and in your favor for the betterment of your future. The pain you feel now will help later if you:\n\n1. Look at what’s not serving you\n2. Listen to your intuition\n3. Learn the meaning behind it\n4. Let go of limits that hold you back
2                                                                                                                                                                                                               @emilsterndorff @awilkinson @supercast @thecapital_io Love hearing that congrats!!
3                                                                                                                              

<IPython.core.display.Javascript object>

After further inspection of the content column, tweets that start with @ are specific tweets to other user. These tweets will be removed becuase we want motinational tweets that are geared towards a general audience. We also have to remove tweets that have hyperlinks since those tweets are more calls to action and not general motivational advise. We will also have to deal with emojis.  